In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

In [4]:
# Initialize WebDriver
driver = webdriver.Chrome()
url = "https://www.sunsirs.com/CommodityPrice.html"
driver.get(url)


In [5]:
def fetch_prices(industry, click_xpath, table_xpath):
    """Clicks on an industry tab and scrapes the related table."""
    try:
        # Click the industry tab
        WebDriverWait(driver, 20).until(
            EC.element_to_be_clickable((By.XPATH, click_xpath))
        ).click()
        time.sleep(2)  # Allow time for content to load

        # Wait for the table to be visible
        WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.XPATH, table_xpath))
        )

        # Find the table
        table = driver.find_element(By.XPATH, table_xpath)

        # Extract rows
        rows = table.find_elements(By.TAG_NAME, "li")  # Table rows are in `<li>`

        # Extract data
        data = []
        for row in rows:
            cols = row.text.strip().split("\n")  # Split text into columns
            if len(cols) >= 4:  # Ensure valid row format
                data.append([industry] + cols[:4])  # Include industry name

        # Convert to DataFrame
        df = pd.DataFrame(data, columns=["Industry", "Product", "Latest Price", "Change", "Unit"])
        return df

    except Exception as e:
        print(f"⚠️ Error fetching {industry} data:", e)
        return pd.DataFrame()  # Return empty DataFrame if error

# Define XPaths
energy_click_xpath = "//*[@id='zwd_body']/div[2]/div/div[2]/div[1]/div/div[2]/div[1]"
steel_click_xpath = "//*[@id='zwd_body']/div[2]/div/div[2]/div[1]/div/div[2]/div[6]"

energy_table_xpath = "//*[@id='zwd_body']/div[2]/div/div[2]/div[2]/div/div[3]/ul"
steel_table_xpath = "//*[@id='zwd_body']/div[2]/div/div[2]/div[2]/div/div[3]/ul"

# Scrape Energy & Steel Data
df_energy = fetch_prices("Energy", energy_click_xpath, energy_table_xpath)
df_steel = fetch_prices("Steel", steel_click_xpath, steel_table_xpath)

# Merge both tables
df_merged = pd.concat([df_energy, df_steel], ignore_index=True)

# Save to Excel
merged_excel_path = r"C:\Users\HP\Downloads\merged_prices.xlsx"
df_merged.to_excel(merged_excel_path, index=False, engine="openpyxl")

# Display Merged Table
print("\n🔹 Merged Energy & Steel Prices Table")
print(df_merged)

# Close the browser
driver.quit()



🔹 Merged Energy & Steel Prices Table
   Industry                Product Latest Price     Change       Unit
0    Energy              Commodity      Sectors      02-11      02-12
1    Energy         Petroleum coke       Energy   2,737.50   2,850.00
2    Energy               Fuel Oil       Energy   5,584.00   5,624.00
3    Energy                 Diesel       Energy   6,978.60   6,996.00
4    Energy                   Coke       Energy   1,660.00   1,660.00
5    Energy                    LPG       Energy   4,930.00   4,930.00
6    Energy           Thermal Coal       Energy     795.00     795.00
7    Energy            Coking coal       Energy   1,537.25   1,537.25
8    Energy               Gasoline       Energy   8,837.60   8,811.60
9     Steel              Commodity      Sectors      02-11      02-12
10    Steel      Cold rolled sheet        Steel   4,205.00   4,207.50
11    Steel  Stainless Steel Plate        Steel  12,164.29  12,171.43
12    Steel       Mild steel plate        Steel   3,